In [1]:
import random
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import openai
# Add the path to the constants file to the system path
sys.path.append('../../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, '../data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'ml-1m/merged_data.dat')
print(f'Data path: {data_path}')

# output

CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user.dat')
print(f'Data path: {CF_OUTPUT_PATH}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user.dat')
print(f'Data path: {CF_RERUN_PATH}')

CF_OUTPUT_TIMESTAMP_PATH = os.path.join(DATA_DIR, 'ml-1m/output/split_timestamp/timestamp_large_CF_fewshot_output_path_ratings_per_user.csv')
print(f'Data path: {CF_OUTPUT_TIMESTAMP_PATH}')

CF_RERUN_TIMESTAMP_PATH = os.path.join(DATA_DIR, 'ml-1m/output/split_timestamp/rerun_timestamp_large_CF_fewshot_output_path_ratings_per_user.csv')
print(f'Data path: {CF_RERUN_TIMESTAMP_PATH}')

# Constants for column names
USER_COLUMN_NAME = 'UserID'
TITLE_COLUMN_NAME = 'Title'
ITEM_ID_COLUMN = 'MovieID'
RATING_COLUMN_NAME = 'Rating'
TIME_STAMP_COLUMN_NAME = 'Timestamp'
SYSTEM_CONTENT = MOVIELENS_CONTENT_SYSTEM


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/merged_data.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/output/CF_fewshot_output_path_ratings_per_user.dat
Data pat

In [2]:
data = pd.read_csv(data_path)
data.head(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama


In [3]:
# Create User-Item Interaction Matrix
interaction_matrix = pd.pivot_table(data, index=USER_COLUMN_NAME, columns=ITEM_ID_COLUMN, values=RATING_COLUMN_NAME).fillna(0)
csr_interaction_matrix = csr_matrix(interaction_matrix.values)

interaction_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
csr_interaction_matrix

<6040x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [5]:
import numpy as np

EPSILON_CONSTANT = 1e-9

def pearson_correlation(interaction_matrix, epsilon_constant=EPSILON_CONSTANT):
    """
    Compute the Pearson Correlation Coefficient matrix for the user-item interaction matrix.

    Args:
        interaction_matrix (numpy.ndarray): A 2D array where rows represent users and columns represent items.
                                             The values in the array are the ratings given by users to items.

    Returns:
        numpy.ndarray: A 2D array representing the Pearson Correlation Coefficients between each pair of users.
    """
    
    # Get the number of users
    n_users = interaction_matrix.shape[0]
    
    # Initialize the Pearson Correlation matrix
    np_user_pearson_corr = np.zeros((n_users, n_users))
    
    print("Starting user-user Pearson Correlation computation...")

    # Iterate over each pair of users
    for i, user_i_vec in enumerate(interaction_matrix):
        for j, user_j_vec in enumerate(interaction_matrix):

            # if i % 50 == 0 and j % 50 == 0:  # Reduce the frequency of print statements
            #     print(f"Processing correlation between users {i} and {j}")

            # Ratings corated by the current pair of users
            mask_i = user_i_vec > 0
            mask_j = user_j_vec > 0
            
            corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))

            if len(corrated_index) == 0:
                continue
            
            # Average value of user_i_vec and user_j_vec
            mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + epsilon_constant)
            mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + epsilon_constant)
            
            # Compute Pearson correlation
            user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
            user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j
            
            r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
            r_uj_sub_r_j_sq = np.square(user_j_sub_mean)
            
            r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
            r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))
            
            sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + epsilon_constant)
            
            np_user_pearson_corr[i][j] = sim

    print("User-user Pearson Correlation computation completed.")
    return np_user_pearson_corr


In [6]:

DELTA = 25
EPSILON = 1e-9

def item_pearson_correlation(interaction_matrix):
    """
    Compute the Pearson Correlation Coefficient matrix for the item-item interaction matrix with significance weighting.

    Args:
        interaction_matrix (2D numpy array): A matrix where rows represent users and columns represent items.
                                              The values in the matrix are the ratings given by users to items.

    Returns:
        numpy.ndarray: A 2D array representing the Pearson Correlation Coefficients between each pair of items.
    """

    n_items = interaction_matrix.shape[1]  # Number of items
    np_item_pearson_corr = np.zeros((n_items, n_items))  # Initialize the Pearson Correlation matrix

    print("Starting item-item Pearson Correlation computation...")

    for i, item_i_vec in enumerate(interaction_matrix.T):
        for j, item_j_vec in enumerate(interaction_matrix.T):

            # if i % 50 == 0 and j % 50 == 0:  # Reduce the frequency of print statements
            #     print(f"Processing correlation between items {i} and {j}")

            # Ratings co-rated by the current pair of items
            mask_i = item_i_vec > 0
            mask_j = item_j_vec > 0

            corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
            if len(corrated_index) == 0:
                # print(f"No corrated ratings for items {i} and {j}. Skipping...")
                continue

            mean_item_i = np.sum(item_i_vec) / (np.sum(np.clip(item_i_vec, 0, 1)) + EPSILON)
            mean_item_j = np.sum(item_j_vec) / (np.sum(np.clip(item_j_vec, 0, 1)) + EPSILON)

            # print(f"Mean rating for item {i}: {mean_item_i}, item {j}: {mean_item_j}")

            item_i_sub_mean = item_i_vec[corrated_index] - mean_item_i
            item_j_sub_mean = item_j_vec[corrated_index] - mean_item_j

            r_ui_sub_ri_sq = np.square(item_i_sub_mean)
            r_uj_sub_rj_sq = np.square(item_j_sub_mean)

            r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
            r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

            sim = np.sum(item_i_sub_mean * item_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

            weighted_sim = (min(len(corrated_index), DELTA) / DELTA) * sim

            # print(f"Correlation between item {i} and item {j}: {weighted_sim}")

            np_item_pearson_corr[i][j] = weighted_sim

    print("Item-item Pearson Correlation computation completed.")
    return np_item_pearson_corr


In [7]:
# %%
# Compute the user-user Pearson Correlation Coefficient Matrix
dense_interaction_matrix = csr_interaction_matrix.toarray()
user_pcc_matrix = pearson_correlation(dense_interaction_matrix)
print(f'User PCC Matrix:\n{user_pcc_matrix}\n')



Starting user-user Pearson Correlation computation...


In [ ]:
# Compute the item-item Pearson Correlation Coefficient Matrix
# Assuming the function 'item_pearson_correlation' takes a dense matrix as input.
# If it still takes a csr_matrix, then convert it inside the function.
item_pcc_matrix = item_pearson_correlation(dense_interaction_matrix.T)
print(f'Item PCC Matrix:\n{item_pcc_matrix}\n')


# Split Random

In [ ]:
%%time

results_df = predict_ratings_with_CF_item_PCC_and_save(
    data=data, 
    user_pcc_matrix=user_pcc_matrix, 
    item_pcc_matrix=item_pcc_matrix,
    user_column_name=USER_COLUMN_NAME, 
    movie_column_name=TITLE_COLUMN_NAME, 
    movie_id_column=ITEM_ID_COLUMN,
    rating_column_name=RATING_COLUMN_NAME, 
    num_ratings_per_user=NUM_RATINGS_PER_USER, 
    num_similar_users=NUM_SIMILAR_USERS,
    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
    test_selection_method='random',
    save_path=CF_OUTPUT_PATH, 
    seed=RANDOM_STATE
)



In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_OUTPUT_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

# Split by Timestamp

In [ ]:
%%time

results_df = predict_ratings_with_CF_item_PCC_and_save(
    data=data, 
    user_pcc_matrix=user_pcc_matrix, 
    item_pcc_matrix=item_pcc_matrix,
    user_column_name=USER_COLUMN_NAME, 
    movie_column_name=TITLE_COLUMN_NAME, 
    timestamp_column_name=TIME_STAMP_COLUMN_NAME,
    movie_id_column=ITEM_ID_COLUMN,
    rating_column_name=RATING_COLUMN_NAME, 
    num_ratings_per_user=NUM_RATINGS_PER_USER, 
    num_similar_users=NUM_SIMILAR_USERS,
    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
    test_selection_method='sequential',
    save_path=CF_OUTPUT_TIMESTAMP_PATH, 
    seed=RANDOM_STATE
)



In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_OUTPUT_TIMESTAMP_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)